# Кэш в GigaChain

Здесь мы рассмотрим:
1. Как в GigaChain framework используется кэширование для оптимизации запросов к LLM модели
2. Два разных способа кэширования запросов, In-Memory и SQLite.

Надеемся это поможет вам разобраться зачем и когда стоит использовать кэширование

In [ ]:
!pip install gigachain_community --quiet --upgrade

## Подготавливаем к работе инстанс GigaChat



In [ ]:
import langchain
from langchain.llms import GigaChat

llm = GigaChat(credentials="<Ваши авторизационные данные>", verify_ssl_certs=False)

## 1. In Memory Cache

In [ ]:
from langchain.cache import InMemoryCache

langchain.llm_cache = InMemoryCache()

### Спросим GigaChat вопрос и измерим сколько потребуется LLM модели времени, чтобы ответить на него.

In [ ]:
%%time

llm.predict("Кто ты?")

CPU times: user 44.9 ms, sys: 5.27 ms, total: 50.2 ms
Wall time: 2.83 s


'Я — модель искусственного интеллекта, созданный для помощи людям. Моя задача — отвечать на вопросы и предоставлять информацию по различным темам. Я могу помочь вам с переводом или объяснением сложных концепций, а также предоставить факты и статистику. Если у вас есть какие-либо конкретные запросы или вопросы, пожалуйста, задайте их мне, и я постараюсь помочь вам наилучшим образом.'

#### Как InMemoryCache кэш хранит данные

**source code**: [cache.py](https://github.com/hwchase17/langchain/blob/v0.0.219/langchain/cache.py#L102)
```python
class InMemoryCache(BaseCache):
    """Cache that stores things in memory."""

    def __init__(self) -> None:
        """Initialize with empty cache."""
        self._cache: Dict[Tuple[str, str], RETURN_VAL_TYPE] = {}
```

Это реализация InMemoryCache.
Она хранит данные в своей оперативной памяти

In [ ]:
# Первый элемент tuple в кэше
list(langchain.llm_cache._cache.keys())[0][0]

'[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "\\u041a\\u0442\\u043e \\u0442\\u044b?"}}]'

In [ ]:
# Второй элемент tuple в кэше
list(langchain.llm_cache._cache.keys())[0][1]

'{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "gigachat", "GigaChat"], "kwargs": {"oauth_token": {"lc": 1, "type": "secret", "id": ["GIGA_OAUTH_TOKEN"]}, "verify_ssl": false, "oauth_verify_ssl": false}}---[(\'stop\', None)]'

### Зададим тот же вопрос и получим ответ быстрее

In [ ]:
%%time

llm.predict("Кто ты?")

CPU times: user 1.86 ms, sys: 0 ns, total: 1.86 ms
Wall time: 1.88 ms


'Я — модель искусственного интеллекта, созданный для помощи людям. Моя задача — отвечать на вопросы и предоставлять информацию по различным темам. Я могу помочь вам с переводом или объяснением сложных концепций, а также предоставить факты и статистику. Если у вас есть какие-либо конкретные запросы или вопросы, пожалуйста, задайте их мне, и я постараюсь помочь вам наилучшим образом.'

## 2. SQLite в качестве кэша

In [ ]:
!rm -f .cache.db

In [ ]:
from langchain.cache import SQLiteCache

langchain.llm_cache = SQLiteCache(database_path=".cache.db")

### Зададим один и тот же вопрос дважды и измерим разницу в производительности

In [ ]:
%%time

llm.predict("Кто ты?")

CPU times: user 31.6 ms, sys: 3.01 ms, total: 34.6 ms
Wall time: 1.97 s


'Я — модель искусственного интеллекта, созданный для помощи людям. Моя задача — отвечать на вопросы и предоставлять информацию по различным темам. Я могу создавать тексты, изображения и генерировать идеи. Если у вас есть какие-либо вопросы или нужна помощь, не стесняйтесь обращаться ко мне!'

In [ ]:
%%time

llm.predict("Кто ты?")

CPU times: user 2.8 ms, sys: 884 µs, total: 3.69 ms
Wall time: 3.44 ms


'Я — модель искусственного интеллекта, созданный для помощи людям. Моя задача — отвечать на вопросы и предоставлять информацию по различным темам. Я могу создавать тексты, изображения и генерировать идеи. Если у вас есть какие-либо вопросы или нужна помощь, не стесняйтесь обращаться ко мне!'

### Добавим лишний пробел в вопросе и посмотрим что будет

In [ ]:
%%time

llm.predict("Кто  ты?")

CPU times: user 25.3 ms, sys: 1.75 ms, total: 27 ms
Wall time: 1.75 s


'Я — мультимодальная нейросетевая модель, созданная командой разработчиков из Сбера. Я умею отвечать на вопросы пользователей, поддерживать диалог, создавать тексты разных форматов и стилей, а также генерировать картинки по запросу.'

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///.cache.db")

### **Почему лишний пробел не дает нормально срабатывать кэшу??**
Потому что кэш ищет точное совпадние запроса. Мы делали запрос без лишнего пробела и он сохранился без него, соответсвенно ответ ни в InMemory, ни в SQLite не будет найден

#### Как SQLite хранит данные

**source code**: [cache.py](https://github.com/hwchase17/langchain/blob/v0.0.219/langchain/cache.py#L128)
```python
class FullLLMCache(Base):  # type: ignore
    """SQLite table for full LLM Cache (all generations)."""

    __tablename__ = "full_llm_cache"
    prompt = Column(String, primary_key=True)
    llm = Column(String, primary_key=True)
    idx = Column(Integer, primary_key=True)
    response = Column(String)


class SQLAlchemyCache(BaseCache):
    """Cache that uses SQAlchemy as a backend."""

    def __init__(self, engine: Engine, cache_schema: Type[FullLLMCache] = FullLLMCache):
        """Initialize by creating all tables."""
        self.engine = engine
        self.cache_schema = cache_schema
        self.cache_schema.metadata.create_all(self.engine)
```

Это схема SQL таблицы `full_llm_cache`, которая хранит данные кэша.

In [ ]:
with engine.connect() as connection:
    rs = connection.exec_driver_sql("select * from full_llm_cache")
    print(rs.keys())
    for row in rs:
        print(row)

RMKeyView(['prompt', 'llm', 'idx', 'response'])
('[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "\\u041a\\u0442\\u043e \\u0442\\u044b?"}}]', '{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "gigachat", "GigaChat"], "kwargs": {"oauth_token": {"lc": 1, "type": "secret", "id": ["GIGA_OAUTH_TOKEN"]}, "verify_ssl": false, "oauth_verify_ssl": false}}---[(\'stop\', None)]', 0, '{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "output", "ChatGeneration"], "kwargs": {"message": {"lc": 1, "type": "constructor", "i ... (1709 characters truncated) ... 0431\\u0440\\u0430\\u0449\\u0430\\u0442\\u044c\\u0441\\u044f \\u043a\\u043e \\u043c\\u043d\\u0435!"}}, "generation_info": {"finish_reason": "stop"}}}')
('[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "\\u041a\\u0442\\u043e  \\u0442\\u044b?"}}]', '{"lc": 1, "type": "constructor", "i

## Семантический кэш

Семантический кэш хранит промпты и ответы, и получает запросы из кэша исходя из схожести запросов по смыслу.
Для того, чтобы работать с семантическим кэшем на данный момент нужно использовать OpenAI или другие ембединги, так как на текущий момент у GigaChat'а ембедингов нет.

In [ ]:
!pip install gigachain_community openai tiktoken --quiet --upgrade

### Следуя доке [Redis](https://redis.com/blog/running-redis-on-google-colab/) устанавливаем и запускаем RedisServer на Google Colab.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<Ваш OpenAI токен>"

In [ ]:
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
!pip install redis
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb

!./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes

./
./redis-stack-server-6.2.6-v7/
./redis-stack-server-6.2.6-v7/bin/
./redis-stack-server-6.2.6-v7/bin/redis-benchmark
./redis-stack-server-6.2.6-v7/bin/redis-cli
./redis-stack-server-6.2.6-v7/bin/redis-sentinel
./redis-stack-server-6.2.6-v7/bin/redis-stack-server
./redis-stack-server-6.2.6-v7/bin/redis-check-rdb
./redis-stack-server-6.2.6-v7/bin/redis-check-aof
./redis-stack-server-6.2.6-v7/bin/redis-server
./redis-stack-server-6.2.6-v7/share/
./redis-stack-server-6.2.6-v7/share/RSAL_LICENSE
./redis-stack-server-6.2.6-v7/share/APACHE_LICENSE
./redis-stack-server-6.2.6-v7/lib/
./redis-stack-server-6.2.6-v7/lib/redisgraph.so
./redis-stack-server-6.2.6-v7/lib/redistimeseries.so
./redis-stack-server-6.2.6-v7/lib/rejson.so
./redis-stack-server-6.2.6-v7/lib/redisbloom.so
./redis-stack-server-6.2.6-v7/lib/redisearch.so
./redis-stack-server-6.2.6-v7/etc/
./redis-stack-server-6.2.6-v7/etc/README
./redis-stack-server-6.2.6-v7/etc/redis-stack.conf
./redis-stack-server-6.2.6-v7/etc/redis-stack-se

In [ ]:
import langchain
from langchain.llms import GigaChat

llm = GigaChat(credentials="<Ваши авторизационные данные>", verify_ssl_certs=False)

### Инициализируем семантический Redis кэш с дефолтным score threshold 0.1

In [ ]:
from langchain.cache import RedisSemanticCache
from langchain.embeddings import OpenAIEmbeddings

langchain.llm_cache = RedisSemanticCache(
    redis_url="redis://localhost:6379",
    embedding=OpenAIEmbeddings(),
    score_threshold=0.1,
)

In [ ]:
%%time

llm("Переведи 'сегодня понедельник' на китайский")

CPU times: user 146 ms, sys: 5.61 ms, total: 151 ms
Wall time: 4.12 s


'上午 (Sìpíng)'

Заметьте что, запрос отличается на 1 слово от предыдущего. Кэш вернул старый ответ, так как score threshold 0.2

In [ ]:
%%time

llm("Переведи 'сегодня вторник' на китайский")

CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 696 ms


'上午 (Sìpíng)'

In [ ]:
%%time

llm("Расскажи мне шутку")

CPU times: user 40.8 ms, sys: 6.05 ms, total: 46.9 ms
Wall time: 3.52 s


'Нашла для вас такую шутку: \n\n— Почему блондинки красят корни волос в чёрный цвет? \n— Потому что они не хотят быть как брюнетки!'

In [ ]:
%%time

llm("Расскажи мне 2 шутки")

CPU times: user 8.29 ms, sys: 137 µs, total: 8.43 ms
Wall time: 677 ms


'Нашла для вас такую шутку: \n\n— Почему блондинки красят корни волос в чёрный цвет? \n— Потому что они не хотят быть как брюнетки!'

### Инициализируем семантический Redis кэш с дефолтным score threshold 0.005

In [ ]:
import redis

client = redis.Redis(host="localhost", port=6379)
client.flushall()  # очищаем старый кэш
langchain.llm_cache = RedisSemanticCache(
    redis_url="redis://localhost:6379",
    embedding=OpenAIEmbeddings(),
    score_threshold=0.05,
)

In [ ]:
%%time

llm("Расскажи мне шутку")

CPU times: user 42.8 ms, sys: 8.82 ms, total: 51.6 ms
Wall time: 3.58 s


'Нашла для вас такую шутку: \n\n— А я вчера слона купил! \n— Ну и как? \n— Маленький такой слонёнок, миленький. Назвал его Гошей. Только вот не знаю, что теперь с ним делать…'

*Запрос с лишним пробелом возмется из кэша*

In [ ]:
%%time

llm("Расскажи мне  шутку")

CPU times: user 9.23 ms, sys: 0 ns, total: 9.23 ms
Wall time: 588 ms


'Нашла для вас такую шутку: \n\n— А я вчера слона купил! \n— Ну и как? \n— Маленький такой слонёнок, миленький. Назвал его Гошей. Только вот не знаю, что теперь с ним делать…'

*А вот запрос двух шуток уже пойдет в гигачат*

In [ ]:
%%time

llm("Расскажи мне 2 шутки")

CPU times: user 39 ms, sys: 90 µs, total: 39.1 ms
Wall time: 2.55 s


'Конечно! Вот две хорошие шутки:\n\n1. Почему слоны не играют в карты? Потому что они всегда выигрывают.\n\n2. Как называется человек, который всё забывает? Склеротик. А как называется тот, кто всё помнит? Злопамятный.'

### Глубокое погружение в Redis семантический кэш

#### Ищем запросы в кэше

In [ ]:
langchain.llm_cache._cache_dict

{'cache:54d4d5d60796b5c24e291e4179785fe1': <langchain.vectorstores.redis.base.Redis at 0x7a42d48dae00>}

#### Вручную выполняем поиск запросов

Чем больше запрос похож на другой тем ближе он будет в массиве

In [ ]:
langchain.llm_cache._cache_dict[
    "cache:54d4d5d60796b5c24e291e4179785fe1"
].similarity_search(query="Расскажи мне шутку")

[Document(page_content='Расскажи мне шутку', metadata={'id': 'doc:cache:54d4d5d60796b5c24e291e4179785fe1:587093b63ad44a2daa6c614ccd23dabe', 'return_val': '[{"text": "\\u041d\\u0430\\u0448\\u043b\\u0430 \\u0434\\u043b\\u044f \\u0432\\u0430\\u0441 \\u0442\\u0430\\u043a\\u0443\\u044e \\u0448\\u0443\\u0442\\u043a\\u0443: \\n\\n\\u2014 \\u0410 \\u044f \\u0432\\u0447\\u0435\\u0440\\u0430 \\u0441\\u043b\\u043e\\u043d\\u0430 \\u043a\\u0443\\u043f\\u0438\\u043b! \\n\\u2014 \\u041d\\u0443 \\u0438 \\u043a\\u0430\\u043a? \\n\\u2014 \\u041c\\u0430\\u043b\\u0435\\u043d\\u044c\\u043a\\u0438\\u0439 \\u0442\\u0430\\u043a\\u043e\\u0439 \\u0441\\u043b\\u043e\\u043d\\u0451\\u043d\\u043e\\u043a, \\u043c\\u0438\\u043b\\u0435\\u043d\\u044c\\u043a\\u0438\\u0439. \\u041d\\u0430\\u0437\\u0432\\u0430\\u043b \\u0435\\u0433\\u043e \\u0413\\u043e\\u0448\\u0435\\u0439. \\u0422\\u043e\\u043b\\u044c\\u043a\\u043e \\u0432\\u043e\\u0442 \\u043d\\u0435 \\u0437\\u043d\\u0430\\u044e, \\u0447\\u0442\\u043e \\u0442\\u0435\\u

In [ ]:
langchain.llm_cache._cache_dict[
    "cache:54d4d5d60796b5c24e291e4179785fe1"
].similarity_search(query="Расскажи мне 2 шутки")

[Document(page_content='Расскажи мне 2 шутки', metadata={'id': 'doc:cache:54d4d5d60796b5c24e291e4179785fe1:78dd6824ae7d48c08c85aececc43351c', 'return_val': '[{"text": "\\u041a\\u043e\\u043d\\u0435\\u0447\\u043d\\u043e! \\u0412\\u043e\\u0442 \\u0434\\u0432\\u0435 \\u0445\\u043e\\u0440\\u043e\\u0448\\u0438\\u0435 \\u0448\\u0443\\u0442\\u043a\\u0438:\\n\\n1. \\u041f\\u043e\\u0447\\u0435\\u043c\\u0443 \\u0441\\u043b\\u043e\\u043d\\u044b \\u043d\\u0435 \\u0438\\u0433\\u0440\\u0430\\u044e\\u0442 \\u0432 \\u043a\\u0430\\u0440\\u0442\\u044b? \\u041f\\u043e\\u0442\\u043e\\u043c\\u0443 \\u0447\\u0442\\u043e \\u043e\\u043d\\u0438 \\u0432\\u0441\\u0435\\u0433\\u0434\\u0430 \\u0432\\u044b\\u0438\\u0433\\u0440\\u044b\\u0432\\u0430\\u044e\\u0442.\\n\\n2. \\u041a\\u0430\\u043a \\u043d\\u0430\\u0437\\u044b\\u0432\\u0430\\u0435\\u0442\\u0441\\u044f \\u0447\\u0435\\u043b\\u043e\\u0432\\u0435\\u043a, \\u043a\\u043e\\u0442\\u043e\\u0440\\u044b\\u0439 \\u0432\\u0441\\u0451 \\u0437\\u0430\\u0431\\u044b\\u043

### Вывод

Score threshold ключевой фактор в использовании поиска по схожести с помощью Redis.

## Семантический кэш с GPTCache

### Что такое GPTCache?

Open source проект, предназначенный для построения и хранения семантического кэша из LLM запросов.

Два способа работы:
1. Точное совпадение
2. Совпадение по схожести

GPTCache предназначен для решения следующих задач:
1. Как мы генерируем эмбединги для запросов? (via embedding function)
2. Как мы храним кэш? (via cache store of data manager, such as SQLite, MySQL, and PostgreSQL. More NoSQL databases will be added in the future)
3. Как мы храним и ищем по эмбедингам? (via vector store of data manager, such as FAISS or vector databases such as Milvus. More vector databases and cloud services will be added in the future.)
4. How to determine eviction policy? (LRU or FIFO)
5. Как мы понимаем, что запрос хранится в кэше? (via evaluation function)

Пожалуйста, посмотрите следующий класс для лучшего понимания как решаются эти задачи

```python
class Cache:
   def init(self,
            cache_enable_func=cache_all,
            pre_embedding_func=last_content,
            embedding_func=string_embedding,
            data_manager: DataManager = get_data_manager(),
            similarity_evaluation=ExactMatchEvaluation(),
            post_process_messages_func=first,
            config=Config(),
            next_cache=None,
            **kwargs
            ):
       self.has_init = True
       self.cache_enable_func = cache_enable_func
       self.pre_embedding_func = pre_embedding_func
       self.embedding_func = embedding_func
       self.data_manager: DataManager = data_manager
       self.similarity_evaluation = similarity_evaluation
       self.post_process_messages_func = post_process_messages_func
       self.data_manager.init(**kwargs)
       self.config = config
       self.next_cache = next_cache
```

In [ ]:
!pip install gptcache --quiet

In [ ]:
import langchain
from langchain.llms import GigaChat

llm = GigaChat(credentials="<Ваши авторизационные данные>", verify_ssl_certs=False)

### Точное совпадение

In [ ]:
import hashlib

from gptcache import Cache
from gptcache.adapter.api import init_similar_cache
from gptcache.manager.factory import manager_factory
from gptcache.processor.pre import get_prompt
from langchain.cache import GPTCache


def get_hashed_name(name):
    return hashlib.sha256(name.encode()).hexdigest()


def init_gptcache(cache_obj: Cache, llm: str):
    hashed_llm = get_hashed_name(llm)
    cache_obj.init(
        pre_embedding_func=get_prompt,
        data_manager=manager_factory(manager="map", data_dir=f"map_cache_{hashed_llm}"),
    )


langchain.llm_cache = GPTCache(init_gptcache)

In [ ]:
question = "What is cache eviction policy?"

In [ ]:
%%time

llm(question)

CPU times: user 69.8 ms, sys: 7.14 ms, total: 76.9 ms
Wall time: 4.35 s


'Cache eviction policy (или кэширование) — это стратегия управления памятью, которая позволяет браузеру использовать один и тот же объект или данные несколько раз при обращении к ним. Это достигается путем хранения данных в кэше, который может быть временно удален из памяти браузера, если он больше не нужен. Когда браузер запрашивает данные, которые были сохранены в кэше, они извлекаются из кэша и возвращаются на клиентскую машину. Кэширование помогает ускорить загрузку страницы, так как не требуется каждый раз загружать данные с сервера. Однако, если кэш заполнен или данные, хранящиеся в нем, устарели, то браузер может удалить их из кэша, чтобы освободить память на сервере.'

In [ ]:
%%time

llm(question)

CPU times: user 848 µs, sys: 0 ns, total: 848 µs
Wall time: 857 µs


'Cache eviction policy (или кэширование) — это стратегия управления памятью, которая позволяет браузеру использовать один и тот же объект или данные несколько раз при обращении к ним. Это достигается путем хранения данных в кэше, который может быть временно удален из памяти браузера, если он больше не нужен. Когда браузер запрашивает данные, которые были сохранены в кэше, они извлекаются из кэша и возвращаются на клиентскую машину. Кэширование помогает ускорить загрузку страницы, так как не требуется каждый раз загружать данные с сервера. Однако, если кэш заполнен или данные, хранящиеся в нем, устарели, то браузер может удалить их из кэша, чтобы освободить память на сервере.'

In [ ]:
%%time

llm("What is cache eviction   policy?")

CPU times: user 34.5 ms, sys: 2.65 ms, total: 37.2 ms
Wall time: 3.99 s


'Cache eviction policy (или просто cache eviction) — это стратегия управления кэшем на сервере. Она заключается в том, чтобы удалять старые данные из кэша при необходимости обновления данных в памяти или других областях хранения. \n\nПравильная cache eviction policy должна быть гибкой и учитывать различные сценарии использования приложения. Например, если приложение часто использует один и тот же набор данных, то можно удалить эти данные из кэша для ускорения работы приложения. Однако, если приложение не обновляет данные в течение длительного времени, то может потребоваться удаление старых данных из кэша для предотвращения возможных проблем с памятью или производительностью.\n\nВажно также убедиться, что приложение регулярно проверяет состояние кэша и удаляет устаревшие данные, чтобы избежать утечек памяти и других проблем производительности.'

### Поиск по схожести

In [ ]:
import hashlib

from gptcache import Cache
from gptcache.adapter.api import init_similar_cache
from langchain.cache import GPTCache


def get_hashed_name(name):
    return hashlib.sha256(name.encode()).hexdigest()


def init_gptcache(cache_obj: Cache, llm: str):
    hashed_llm = get_hashed_name(llm)
    init_similar_cache(cache_obj=cache_obj, data_dir=f"similar_cache_{hashed_llm}")


langchain.llm_cache = GPTCache(init_gptcache)

In [ ]:
%%time

llm(question)

CPU times: user 1.41 s, sys: 0 ns, total: 1.41 s
Wall time: 3.28 s


'Cache eviction policy (или "caching policy") — это стратегия кэширования, которая определяет, какие данные будут храниться в кэше и когда они должны быть удалены из памяти. \n\nВ общем случае, cache eviction policy состоит из нескольких шагов:\n\n1. Определение размера кэша: размер кэша определяется на основе требований приложения и доступных ресурсов. Обычно он составляет от нескольких килобайт до нескольких мегабайт.\n\n2. Выбор алгоритма кэширования: существует множество алгоритмов кэширования, таких как LRU (Least Recently Used), Hash Code, Double Counting, RAM-кэш и другие. Каждый алгоритм имеет свои преимущества и недостатки, поэтому выбор конкретного зависит от требований приложения.\n\n3. Установка времени жизни кэша: время жизни кэша может быть установлено на определенный период времени или навсегда. Это позволяет системе периодически очищать кэш для ускорения работы приложений.\n\n4. Удаление данных из кэша: если данные больше не нужны приложению, то их можно удалить из кэша

In [ ]:
%%time

llm(question)

CPU times: user 903 ms, sys: 0 ns, total: 903 ms
Wall time: 915 ms


'Cache eviction policy (или "caching policy") — это стратегия кэширования, которая определяет, какие данные будут храниться в кэше и когда они должны быть удалены из памяти. \n\nВ общем случае, cache eviction policy состоит из нескольких шагов:\n\n1. Определение размера кэша: размер кэша определяется на основе требований приложения и доступных ресурсов. Обычно он составляет от нескольких килобайт до нескольких мегабайт.\n\n2. Выбор алгоритма кэширования: существует множество алгоритмов кэширования, таких как LRU (Least Recently Used), Hash Code, Double Counting, RAM-кэш и другие. Каждый алгоритм имеет свои преимущества и недостатки, поэтому выбор конкретного зависит от требований приложения.\n\n3. Установка времени жизни кэша: время жизни кэша может быть установлено на определенный период времени или навсегда. Это позволяет системе периодически очищать кэш для ускорения работы приложений.\n\n4. Удаление данных из кэша: если данные больше не нужны приложению, то их можно удалить из кэша

In [ ]:
%%time

llm("What is cache eviction   policy?")

CPU times: user 929 ms, sys: 0 ns, total: 929 ms
Wall time: 940 ms


'Cache eviction policy (или "caching policy") — это стратегия кэширования, которая определяет, какие данные будут храниться в кэше и когда они должны быть удалены из памяти. \n\nВ общем случае, cache eviction policy состоит из нескольких шагов:\n\n1. Определение размера кэша: размер кэша определяется на основе требований приложения и доступных ресурсов. Обычно он составляет от нескольких килобайт до нескольких мегабайт.\n\n2. Выбор алгоритма кэширования: существует множество алгоритмов кэширования, таких как LRU (Least Recently Used), Hash Code, Double Counting, RAM-кэш и другие. Каждый алгоритм имеет свои преимущества и недостатки, поэтому выбор конкретного зависит от требований приложения.\n\n3. Установка времени жизни кэша: время жизни кэша может быть установлено на определенный период времени или навсегда. Это позволяет системе периодически очищать кэш для ускорения работы приложений.\n\n4. Удаление данных из кэша: если данные больше не нужны приложению, то их можно удалить из кэша

In [ ]:
%%time

llm("Give me a peach")

CPU times: user 2.76 s, sys: 0 ns, total: 2.76 s
Wall time: 5.48 s


"I'm sorry, I don't have any peaches in my system right now. Can you please try asking for something else?"

In [ ]:
%%time

llm("Give me 2 peaches")

CPU times: user 1.24 s, sys: 0 ns, total: 1.24 s
Wall time: 1.53 s


"I'm sorry, I don't have any peaches in my system right now. Can you please try asking for something else?"